# Demo: Serenity Portfolio Storage API

This notebook demonstrates how to use the Portfolio Storage API to manage your portfolio details.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
from os import getenv
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv('SERENITY_CONFIG_ID', None))

In [ ]:
import datetime

import pandas as pd

# create an alias to the api
api = connect_widget.get_api()

## Create a Portfolio Metadata
A Portfolio Metadata stores common metadata for portfolios.

First retrieve the UUID for USD currency that we'll use as the base currency for the Portfolios

In [ ]:
usd_currency = api.refdata().get_currency("USD")
# Print the output as table
pd.json_normalize(usd_currency.dict())

In [ ]:
from serenity_types.portfolio.core import PortfolioMetadataCreateRequest, Strategy, StrategyType

create_request = PortfolioMetadataCreateRequest(
    base_currency_id=usd_currency.asset_id,
    name="Python SDK Demo Portfolio v1",
    owner="Python SDK",
    strategy=Strategy(
        strategy_type=StrategyType.MARKET_NEUTRAL,
        description="Demo Portfolio Market Neutral Strategy"),
    updated_by="Python SDK",
    tags=["demo", "sdk"]
)
metadata = api.portfolio().create_portfolio_metadata(create_request)
# Print the output as table
pd.json_normalize(metadata.dict())

## List all Portfolio Metadata

In [ ]:

metadata_list = api.portfolio().list_portfolio_metadata()
# Print the output as table
pd.json_normalize((metadata.dict() for metadata in metadata_list))

Or retrieve the Portfolio metadata with the known metadata id

In [ ]:
metadata_id = metadata.metadata_id
metadata = api.portfolio().get_portfolio_metadata(metadata_id)
# Print the output as table
pd.json_normalize([metadata.dict()])

## Update an existing Portfolio Metadata

In [ ]:

from serenity_types.portfolio.core import PortfolioMetadataUpdateRequest

update_request = PortfolioMetadataUpdateRequest(
    base_currency_id=usd_currency.asset_id,
    name="Python SDK Update Demo Portfolio v1",
    owner="Python SDK",
    strategy=Strategy(
        strategy_type=StrategyType.EVENT_DRIVEN,
        description="Demo Portfolio Event Driven Strategy"),
    updated_by="Python SDK",
    tags=["demo", "sdk"]
)
metadata = api.portfolio().update_portfolio_metadata(metadata.metadata_id, update_request)
# Print the output as table
pd.json_normalize([metadata.dict()])

## Create Portfolio Snapshot
A Portfolio Snapshot provides a snapshot of the portfolio's positions at a particular moment in time

First retrieve the UUIDs of the base assets (BTC and ETH)

In [ ]:
tokens = api.refdata().get_tokens(["BTC", "ETH"])
btc_token = tokens['BTC']
eth_token = tokens['ETH']
# Print the output as table
pd.json_normalize([btc_token.dict(), eth_token.dict()])

Retrieve the UUIDs for Future with BTC and ETH as the underlier

In [ ]:
from serenity_types.refdata.asset import AssetSearchRequest, AssetType

derivatives_search_request = AssetSearchRequest(
    asset_types=[AssetType.FUTURE],
    underlier_assets=[btc_token.asset_id, eth_token.asset_id])

tokens = api.refdata().search_assets(derivatives_search_request)

df = pd.DataFrame((asset.dict() for asset in tokens))
btc_future = tokens[df.query("native_symbol.str.contains('BTC')").index[0]]
eth_future = tokens[df.query("native_symbol.str.contains('ETH')").index[0]]
# Print the output as table
pd.json_normalize([btc_future.dict(), eth_future.dict()])

In [ ]:
import random
from serenity_types.ledger.balance import Balance
from serenity_types.portfolio.core import PortfolioSnapshotCreateRequest

create_request = PortfolioSnapshotCreateRequest(
    portfolio_metadata_id=metadata.metadata_id,
    as_of_time=datetime.datetime.now(datetime.timezone.utc),
    updated_by="Python SDK",
    balances=[
        Balance(asset_id=btc_token.asset_id, quantity=random.uniform(1, 10)),
        Balance(asset_id=eth_token.asset_id, quantity=random.uniform(10, 50)),
        Balance(asset_id=btc_future.asset_id, quantity=random.randint(10, 50)),
        Balance(asset_id=eth_future.asset_id, quantity=random.randint(10, 50))
    ]
)
snapshot = api.portfolio().create_portfolio_snapshot(create_request)
# Print the output as table
pd.json_normalize([snapshot.dict()], record_path="balances", meta=['portfolio_metadata_id', 'snapshot_id'])

## Retrieve all the Portfolio Snapshots associated with the Portfolio Metadata

In [ ]:
snapshots = api.portfolio().list_portfolio_snapshots(metadata_id)
# Print the output as table
pd.json_normalize((snapshot.dict() for snapshot in snapshots), record_path="balances", meta=['portfolio_metadata_id', 'snapshot_id', 'as_of_time', 'version'])

## Amend a Portfolio Snapshot

In [ ]:
from serenity_types.portfolio.core import PortfolioSnapshotUpdateRequest

update_request = PortfolioSnapshotUpdateRequest(
    portfolio_metadata_id=metadata.metadata_id,
    as_of_time=snapshot.as_of_time,
    updated_by="Python SDK",
    balances=[
        Balance(asset_id=btc_token.asset_id, quantity=random.uniform(10, 50)),
        Balance(asset_id=eth_token.asset_id, quantity=random.uniform(20, 60)),
        Balance(asset_id=btc_future.asset_id, quantity=random.randint(20, 100)),
        Balance(asset_id=eth_future.asset_id, quantity=random.randint(20, 100))
    ]
)
updated_snapshot = api.portfolio().update_portfolio_snapshot(snapshot.snapshot_id, update_request)
# Print the output as table
pd.json_normalize(updated_snapshot.dict(), record_path="balances", meta=['portfolio_metadata_id', 'snapshot_id', 'as_of_time', 'version'])

## Convert Portfolio Snapshot into a format compatible with the current SDK API

In [ ]:
legacy_portfolio = api.portfolio().to_legacy_portfolio(updated_snapshot)
# Print the output as table
pd.json_normalize(legacy_portfolio.to_asset_positions())

## Delete Portfolio Snapshot

In [ ]:
api.portfolio().delete_portfolio_snapshot(snapshot.snapshot_id)

## Delete Portfolio Metadata

In [ ]:
api.portfolio().delete_portfolio_metadata(metadata.metadata_id)